In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv('train_older.csv')
test=pd.read_csv('test_older.csv')
sample=pd.read_csv('sample_submission.csv')

In [4]:
train.iloc[:,2:].head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,-2.246,1.825,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,0.004,-0.291,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,0.137,0.183,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,0.503,0.274,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,-0.012,2.198,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253


In [7]:
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, train_test_split, GridSearchCV
from sklearn.feature_selection import RFECV,RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [4]:
X=train.iloc[:,2:]
y=train.iloc[:,1]
Xtest=test.iloc[:,1:]

In [23]:
lg=LogisticRegression(solver='liblinear', penalty='l1',C=0.1,class_weight='balanced')
param_grid={'C':[0.001,0.01,0.1,1,10,100,1000], 'class_weight':['balanced',None], 'penalty':['l1','l2']}
gcv=GridSearchCV(lg,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
gcv.fit(X,y)

GridSearchCV(cv=5,
             estimator=LogisticRegression(C=0.1, class_weight='balanced',
                                          penalty='l1', solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             scoring='roc_auc')

In [24]:
gcv.best_params_

{'C': 1, 'class_weight': None, 'penalty': 'l1'}

In [ ]:
lg=LogisticRegression(solver='liblinear', penalty='l1',C=1)
rfe=RFECV(lg,cv=5,scoring='roc_auc',n_jobs=-1)
rfe.fit(X,y)

In [30]:
X.iloc[:,rfe.support_].shape

(250, 44)

In [27]:
#features = [ '16', '33', '43', '45', '52', '63', '65', '73', '90', '91', '117', '133', '134', '149', '189', '199', '217', '237', '258', '295']
features=X.iloc[:, rfe.support_].columns

folds=RepeatedStratifiedKFold(n_splits = 5, n_repeats = 50, random_state = 42)
preds=[]
params = {'random_state':300, 'n_jobs':-1, 'C':1, 'penalty':'l1', 'solver':'liblinear','verbose':0}
for train_id, val_id in folds.split(X,y):
    train_x, train_y = X[features].iloc[train_id], y.iloc[train_id]
    val_x, val_y = X[features].iloc[val_id], y.iloc[val_id]
    clf = LogisticRegression(**params)
    clf.fit(train_x, train_y)
    predict=clf.predict_proba(Xtest[features])[:,1:]
    preds.append(predict)
preds=np.concatenate(preds,axis=1)

/home/joao/anaconda3/envs/python3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
/home/joao/anaconda3/envs/python3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
/home/joao/anaconda3/envs/python3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
/home/joao/anaconda3/envs/python3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1523: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
/home/joao/anaconda3/envs/python3.9/lib/python3.9/site-packages/sklearn/linear_model/_lo

In [32]:
p=gcv.predict_proba(Xtest)[:,1]

In [33]:
sample['target']=p
sample.to_csv('submission.csv',index=False)